In [1]:
import tensorflow_hub as hub
import tensorflow as tf

import tensorflow_text as text


In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [5]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/new one/nlp/spam_ham_dataset.csv')
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [7]:
df.groupby('label').describe()
df = df.drop('Unnamed: 0', axis=1)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df['text'],df['label_num'],test_size=0.2,stratify = df['label_num'])

In [10]:
bert_preprocess_model=hub.KerasLayer(preprocess_url)

dict_keys(['input_word_ids', 'input_type_ids', 'input_mask'])

In [13]:
bert_model = hub.KerasLayer(encoder_url)


In [20]:
#functional model
text_input=tf.keras.layers.Input(shape=(),dtype=tf.string,name='text')
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocessed_text)

l=tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])

l=tf.keras.layers.Dense(1, activation='sigmoid', name = 'output')(l)

model = tf.keras.Model(inputs=[text_input],outputs=[l])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [22]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])  

In [23]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
130/130 [==============================] - 1839s 14s/step - loss: 0.5260 - accuracy: 0.7548
Epoch 2/5
130/130 [==============================] - 1810s 14s/step - loss: 0.4453 - accuracy: 0.8119
Epoch 3/5
130/130 [==============================] - 1815s 14s/step - loss: 0.3969 - accuracy: 0.8450
Epoch 4/5
130/130 [==============================] - 1819s 14s/step - loss: 0.3588 - accuracy: 0.8590
Epoch 5/5
130/130 [==============================] - 1815s 14s/step - loss: 0.3315 - accuracy: 0.8777


In [24]:
model.evaluate(X_test,y_test)

33/33 [==============================] - 469s 14s/step - loss: 0.3082 - accuracy: 0.8899


[0.308152973651886, 0.8898550868034363]

In [25]:
samples=[
         'aGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
]

In [27]:
model.predict(samples)

array([[0.59501356]], dtype=float32)

In [29]:
name = 'email'
saved_model_path='/content/gdrive/MyDrive/new one/nlp/{}_bert'.format(name.replace('/','_'))
model.save(saved_model_path)


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/new one/nlp/email_bert/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/new one/nlp/email_bert/assets


In [30]:
reload_model = tf.saved_model.load('/content/gdrive/MyDrive/new one/nlp/email_bert')

In [32]:
reload_model(tf.constant(samples))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.59501356]], dtype=float32)>